# Défis open data : JOP2024 et offre culturelle

In [113]:
import json

import pandas as pd

## Jeu de données : sites de compétition

In [114]:
with open("datasets/raw/paris-2024-sites-de-competition.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.json_normalize(data)

In [115]:
df.head(3)

,code_site,nom_site,category_id,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
0,ALX,Pont Alexandre III,venue-olympic,Cyclisme sur route - arrivée Contre-la-montre ...,2024-07-27,2024-08-09,None,"48,8637","2,3134",2.313400,48.86370
1,BCY,Arena Bercy,venue-olympic,"Basketball (BKB), Gymnastique artistique (GAR)...",2024-07-27,2024-08-11,None,"48,83863","2,378597",2.378597,48.83863
2,CDM,Arena Champ de Mars,venue-olympic,"Judo (JUD), Lutte (WRE)",2024-07-27,2024-08-11,None,"48,8531","2,30252",2.302520,48.85310


In [116]:
df.sample(3)

,code_site,nom_site,category_id,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
40,TRO,Trocadéro,venue-olympic,"Athlétisme marche (ATW), Cyclisme sur route - ...",2024-08-01,2024-08-07,None,"48,85972558","2,29221884",2.292219,48.859726
44,BCY,Arena Bercy,venue-paralympic,Basketball fauteuil (PWBK),2024-08-29,2024-09-08,None,"48,83863","2,378597",2.378597,48.838630
49,LBO,Site d’escalade du Bourget,venue-olympic,Escalade (CLB),2024-08-05,2024-08-10,None,"48,93693402","2,41997931",2.419979,48.936934


In [117]:
df.shape

(63, 11)

In [118]:
df.columns

Index(['code_site', 'nom_site', 'category_id', 'sports', 'start_date',
       'end_date', 'adress', 'latitude', 'longitude', 'point_geo.lon',
       'point_geo.lat'],
      dtype='object')

In [119]:
df.dtypes

code_site         object
nom_site          object
category_id       object
sports            object
start_date        object
end_date          object
adress            object
latitude          object
longitude         object
point_geo.lon    float64
point_geo.lat    float64
dtype: object

In [120]:
df.nunique()

code_site        44
nom_site         44
category_id       2
sports           56
start_date       15
end_date         18
adress            0
latitude         43
longitude        43
point_geo.lon    43
point_geo.lat    43
dtype: int64

In [121]:
df.isna().sum()

code_site         0
nom_site          0
category_id       0
sports            0
start_date        0
end_date          0
adress           63
latitude          0
longitude         0
point_geo.lon     0
point_geo.lat     0
dtype: int64

La colonne `code_site`ne sera pas utile, on la supprime.

In [122]:
df = df.drop(columns="code_site")

df.sample(3)

,nom_site,category_id,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
16,Stade Roland Garros,venue-paralympic,Tennis fauteuil (PWTE),2024-08-30,2024-09-07,None,"48,845968","2,253522",2.253522,48.845968
43,Pont Alexandre III,venue-paralympic,Para Triathlon (PTRI),2024-09-01,2024-09-02,None,"48,8637","2,3134",2.313400,48.863700
30,Colline d’Elancourt,venue-olympic,Cyclisme VTT (MTB),2024-07-28,2024-07-29,None,"48,78981063","1,9642379",1.964238,48.789811


`category_id` n'a que deux valeurs, on supprime le prefix "venue-" et on renomme cette colonne en `games_type`.

In [123]:
df["category_id"] = df["category_id"].str.removeprefix("venue-")
df = df.rename(columns={"category_id": "games_type"})

df.sample(3)

,nom_site,games_type,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
39,Stade de France,olympic,"Athlétisme (ATH), Rugby à 7 (RU7), Cérémonie d...",2024-07-24,2024-08-11,None,"48,924475","2,360127",2.360127,48.924475
32,La Concorde 1,olympic,"Basketball 3x3 (BK3), Breaking (BKG)",2024-07-30,2024-08-10,None,"48,86640642","2,32119515",2.321195,48.866406
11,Marina de Marseille,olympic,Voile (SAL),2024-07-28,2024-08-08,None,"43,2661","5,3678",5.367800,43.266100


`sports`, semble contenir plusieurs sports pour une même ligne.

In [124]:
df["sports"].unique()[:20]

array(['Cyclisme sur route - arrivée Contre-la-montre (CRD), Natation marathon (OWS), Triathlon (TRI)',
       'Basketball (BKB), Gymnastique artistique (GAR), Gymnastique trampoline (GTR)',
       'Judo (JUD), Lutte (WRE)',
       'Badminton (BDM), Gymnastique rythmique (GRY)',
       'Para badminton (PBDM), Para Haltérophilie (PPWL)',
       'Para cyclisme sur route (PCRD)', 'Para Tir Sportif (PSHO)',
       'Para Natation (PSWM)', 'Cécifoot (PFBB)',
       'Escrime (FEN), Taekwondo (TKW)',
       'Athlétisme - départ Marathon (ATM)', 'Voile (SAL)',
       'Football (FBL)', 'Boxe (BOX), Pentathlon moderne (MDN)',
       'Volleyball assis (PVBS)', 'Tennis fauteuil (PWTE)',
       'Tennis (TEN), Boxe (BOX)', 'Volley-ball (VVO)', 'Boccia (PBOC)',
       'Surf (SRF)'], dtype=object)

Il y a 1, 2, 3 sports ou plus pour une même ligne mais on laisse comme cela. Par contre, on supprime le code du sport qui se trouve entre parenthèses.

In [125]:
df["sports"] = df["sports"].replace(r"\s*\([^)]*\)", "", regex=True)

df.sample(3)

,nom_site,games_type,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
40,Trocadéro,olympic,"Athlétisme marche, Cyclisme sur route - Course...",2024-08-01,2024-08-07,None,"48,85972558","2,29221884",2.292219,48.859726
21,Vélodrome National de Saint-Quentin-en-Yvelines,paralympic,Para Cyclisme sur piste,2024-08-29,2024-09-01,None,"48,78800979",2.03498269,2.034983,48.788010
20,Tahiti Teahupo'o,olympic,Surf,2024-07-27,2024-07-30,None,"-17,86693","-149,25208",-149.252080,-17.866930


Les dates ne sont pas au bon format, essayons de les convertir pour voir si aucune ne pose problème.

In [126]:
df["start_date"] = pd.to_datetime(df["start_date"])
df["end_date"] = pd.to_datetime(df["end_date"])

df[["start_date", "end_date"]].dtypes

start_date    datetime64[ns]
end_date      datetime64[ns]
dtype: object

La conversion a fonctionné sans erreur.

`adress`est toujours vide, on supprime la colonne.

In [127]:
df = df.drop(columns="adress")

`latitude` et `longitude` ne sont pas gardées :
* en doublon avec `point_geo.lon` et `point_geo.lat` ;
* pas au format numérique.

In [128]:
df = df.drop(columns=["latitude", "longitude"])

df = df.rename(columns={"point_geo.lon": "lon", "point_geo.lat": "lat"})

df.sample(3)

,nom_site,games_type,sports,start_date,end_date,lon,lat
42,Stade BMX de Saint-Quentin-en-Yvelines,olympic,BMX Racing,2024-09-01,2024-09-02,2.034983,48.788010
27,Arena Champ de Mars,paralympic,"Para Judo, Rugby fauteuil",2024-08-29,2024-09-07,2.302520,48.853100
30,Colline d’Elancourt,olympic,Cyclisme VTT,2024-07-28,2024-07-29,1.964238,48.789811


Le dataset est prêt à être exporté.

In [129]:
# df.to_json("datasets/jop2024-competition-sites.json", orient="index", indent=4, date_format="iso")

## Jeu de données : événements olympiade culturelle

In [131]:
with open("datasets/raw/paris-2024-evenements-olympiade-culturelle.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.json_normalize(data)

In [132]:
df.head(3)

,project_oc_r_id,id,name,nom_de_la_structure_c,presentation_synthetique_du_projet_c,date_de_debut_c,date_de_fin_c,public_adresse_c,lieu_de_presentation_c,adresse_c,...,commune,tarif_du_projet_c,veuillez_preciser_les_tarifs_du_projet_c,instagram_link,facebook_link,twitter_link,discipline_principale_du_projet_c,geolocation.lon,geolocation.lat,geolocation
0,a7e2o0000000P6GAAU,a7f2o00000006cFAAQ,Concrete Block Party,Rara Woulib,"""Riding Block Party"" est une installation urba...",2024-05-25T12:00:00+00:00,2024-05-25T19:00:00+00:00,Tous publics,Place Villeneuve-Bargemon,"Place Villeneuve - Bargemon, 13002 Marseille, ...",...,Marseille,Gratuit,None,https://www.instagram.com/cie_rarawoulib/,https://www.facebook.com/cierarawoulib/,None,[Arts de la rue],5.370179,43.296731,NaN
1,a7e2o0000000P6LAAU,a7f2o00000006cKAAQ,Les Coureurs,Barroco Théâtre,"""Les marathoniens sont des animaux étranges qu...",2023-09-03T07:00:00+00:00,2023-09-03T09:30:00+00:00,Tous publics,St Etienne de Chigny,"37230 Saint-Étienne-de-Chigny, France",...,Saint-Étienne-de-Chigny,Gratuit,None,None,https://www.facebook.com/barrocotheatre37,None,[Théâtre],0.526444,47.383122,NaN
2,a7e2o0000000PBRAA2,a7f2o00000008NZAAY,L'Art du Sport à Saint-Cloud - Au delà de la c...,Espace Clodoaldien Loisirs Animation,"Très souvent, la photo se passe de légende. El...",2024-02-05T07:00:00+00:00,2024-04-07T09:39:00+00:00,Tous publics,Le Carré,"3 Bis Rue d'Orléans, 92210 Saint-Cloud, France",...,Saint-Cloud,Gratuit,None,None,None,None,[Photographie],2.218907,48.840286,NaN


In [133]:
df.sample(3)

,project_oc_r_id,id,name,nom_de_la_structure_c,presentation_synthetique_du_projet_c,date_de_debut_c,date_de_fin_c,public_adresse_c,lieu_de_presentation_c,adresse_c,...,commune,tarif_du_projet_c,veuillez_preciser_les_tarifs_du_projet_c,instagram_link,facebook_link,twitter_link,discipline_principale_du_projet_c,geolocation.lon,geolocation.lat,geolocation
2898,a7e2o000000LIKJAA4,a7f2o000000LidLAAS,Jazz des cinq continents,Conservatoire Charles Munch - CMA11,Le département Jazz du conservatoire Charles M...,2023-06-23T17:00:00+00:00,2023-06-23T19:00:00+00:00,Tous publics,maison des métallos,94 Rue Jean-Pierre Timbaud,...,Paris,Gratuit,None,None,None,None,[Musique],2.377780,48.867697,NaN
3164,a7e2o000000PDKBAA4,a7f2o000000PchjAAC,Les orchestres à l'école accompagnent la flamme,Association orchestre à l'école,"Du 8 mai au 24 juillet 2024, la Flamme Olympiq...",2024-06-04T09:30:00+00:00,2024-06-04T10:30:00+00:00,Tous publics,Espace public,"Rue du Maréchal Joffre, 85000 La Roche-sur-Yon...",...,La Roche-sur-Yon,Gratuit,None,https://www.instagram.com/orchestrealecole.ass...,https://www.facebook.com/OrchestrealEcole/?loc...,https://x.com/Orchestreecole?ref_src=twsrc%5Eg...,[Musique],-1.425095,46.667445,NaN
1234,a7e2o0000004LrtAAE,a7f2o0000004U72AAE,Visite-atelier en famille,Musée Dauphinois,Visites-ateliers en famille\nUne visite spécia...,2024-08-21T13:00:00+00:00,2024-08-21T14:30:00+00:00,Jeune public,Musée dauphinois,"30 Rue Maurice Gignoux, 38000 Grenoble, France",...,Grenoble,Payant,"3,80 Euros par enfant",https://www.instagram.com/museedauphinois/,https://www.facebook.com/museedauphinois?ref=hl#,None,[Arts visuels/plastiques],5.726660,45.194946,NaN


In [134]:
df.shape

(4262, 23)

In [135]:
df.columns

Index(['project_oc_r_id', 'id', 'name', 'nom_de_la_structure_c',
       'presentation_synthetique_du_projet_c', 'date_de_debut_c',
       'date_de_fin_c', 'public_adresse_c', 'lieu_de_presentation_c',
       'adresse_c', 'longitude_c', 'latitude_c', 'departement', 'commune',
       'tarif_du_projet_c', 'veuillez_preciser_les_tarifs_du_projet_c',
       'instagram_link', 'facebook_link', 'twitter_link',
       'discipline_principale_du_projet_c', 'geolocation.lon',
       'geolocation.lat', 'geolocation'],
      dtype='object')

In [136]:
df.dtypes

project_oc_r_id                              object
id                                           object
name                                         object
nom_de_la_structure_c                        object
presentation_synthetique_du_projet_c         object
date_de_debut_c                              object
date_de_fin_c                                object
public_adresse_c                             object
lieu_de_presentation_c                       object
adresse_c                                    object
longitude_c                                  object
latitude_c                                   object
departement                                 float64
commune                                      object
tarif_du_projet_c                            object
veuillez_preciser_les_tarifs_du_projet_c     object
instagram_link                               object
facebook_link                                object
twitter_link                                 object
discipline_p

In [137]:
df.isna().sum()

project_oc_r_id                                0
id                                             0
name                                           0
nom_de_la_structure_c                          0
presentation_synthetique_du_projet_c          47
date_de_debut_c                                0
date_de_fin_c                                  0
public_adresse_c                               0
lieu_de_presentation_c                         2
adresse_c                                      1
longitude_c                                   57
latitude_c                                    57
departement                                   92
commune                                       89
tarif_du_projet_c                            174
veuillez_preciser_les_tarifs_du_projet_c    3147
instagram_link                              1979
facebook_link                               1575
twitter_link                                3133
discipline_principale_du_projet_c              0
geolocation.lon     

In [138]:
print(
    f"nunique() :\n"
    f"- project_oc_r_id : {df['project_oc_r_id'].nunique()}\n"
    f"- id : {df['id'].nunique()}\n"
    f"- name : {df['name'].nunique()}"
)

nunique() :
- project_oc_r_id : 2493
- id : 4262
- name : 2420


Le nombre d'`id` correspond au nombre d'enregistrements mais ce n'est ni le cas de `project_oc_r_id`ou `name`.

In [139]:
df["name"].value_counts()

name
Atelier pop-up Totems sports                                              18
BOOST                                                                     15
Expositions "Art et Sport"                                                14
Tour de Provence culturel en e-vélo                                       10
13 en Jeux                                                                10
                                                                          ..
L’exposition des Affiches Artistiques de Paris 2024 à L'Aigle              1
L’exposition des Affiches Artistiques de Paris 2024 à Paris 20             1
L’exposition des Affiches Artistiques de Paris 2024 au lycée Vauquelin     1
Exposition des affiches artistiques de Paris 2024 à Blanche Paris          1
Nuit Blanche                                                               1
Name: count, Length: 2420, dtype: int64

In [140]:
df.loc[df["name"] == "BOOST"][:3]

,project_oc_r_id,id,name,nom_de_la_structure_c,presentation_synthetique_du_projet_c,date_de_debut_c,date_de_fin_c,public_adresse_c,lieu_de_presentation_c,adresse_c,...,commune,tarif_du_projet_c,veuillez_preciser_les_tarifs_du_projet_c,instagram_link,facebook_link,twitter_link,discipline_principale_du_projet_c,geolocation.lon,geolocation.lat,geolocation
15,a7e2o0000000PEaAAM,a7f2o00000006foAAA,BOOST,Rencontres Chorégraphiques Internationales de ...,La saison BOOST commencée en septembre 2023 à ...,2024-04-28T11:30:00+00:00,2024-04-28T16:00:00+00:00,Tous publics,CGT de Montreuil,"263 Rue de Paris, 93100 Montreuil, France",...,Montreuil,Payant,"Réservez vos places sur la billetterie, direct...",https://www.instagram.com/lesrencontreschoregr...,https://www.facebook.com/rencontres.choregraph...,None,[Danse],2.416288,48.854348,NaN
508,a7e2o0000000R35AAE,a7f2o00000008LyAAI,BOOST,Rencontres Chorégraphiques Internationales de ...,La saison BOOST commencée en septembre 2023 à ...,2024-05-03T17:30:00+00:00,2024-05-03T18:00:00+00:00,Tous publics,Parvis du théâtre du Garde-Chasse,"181 bis Rue de Paris, 93260 Les Lilas, France",...,Les Lilas,Payant,"Réservez vos places sur la billetterie, direct...",https://www.instagram.com/rencontreschoregraph...,https://www.facebook.com/rencontres.choregraph...,None,[Danse],2.420070,48.881069,NaN
510,a7e2o0000000R35AAE,a7f2o00000008M0AAI,BOOST,Rencontres Chorégraphiques Internationales de ...,La saison BOOST commencée en septembre 2023 à ...,2024-05-04T12:00:00+00:00,2024-05-04T21:00:00+00:00,Tous publics,Théâtre du Fil de l'Eau,"20 Rue Delizy, 93500 Pantin, France",...,Pantin,Payant,"Réservez vos places sur la billetterie, direct...",https://www.instagram.com/rencontreschoregraph...,https://www.facebook.com/rencontres.choregraph...,None,[Danse],2.411070,48.895346,NaN


Un même événement peu avoir lieu plusieurs fois, soit au même endroit à des horaires différents ou alors à d'autres endroits. On va supprimer les deux colonnes relatives aux id car elles ne nous servirons pas.

In [141]:
df = df.drop(columns=["project_oc_r_id", "id"])

In [142]:
try:
    df.nunique()
except Exception as e:
    print(e)

unhashable type: 'list'


Une des colonnes contient des valeurs considérées comme une liste.

In [143]:
print(df.iloc[0])

name                                                                     Concrete Block Party
nom_de_la_structure_c                                                             Rara Woulib
presentation_synthetique_du_projet_c        "Riding Block Party" est une installation urba...
date_de_debut_c                                                     2024-05-25T12:00:00+00:00
date_de_fin_c                                                       2024-05-25T19:00:00+00:00
public_adresse_c                                                                 Tous publics
lieu_de_presentation_c                                              Place Villeneuve-Bargemon
adresse_c                                   Place Villeneuve - Bargemon, 13002 Marseille, ...
longitude_c                                                                 5.370179450940349
latitude_c                                                                 43.296730600000004
departement                                                 

Il s'agit de la colonne `discipline_principale_du_projet_c`. On la convertit en `string`et on supprime les caratères inutiles.

In [144]:
df["discipline_principale_du_projet_c"] =  df["discipline_principale_du_projet_c"].astype("string")
df["discipline_principale_du_projet_c"] = df["discipline_principale_du_projet_c"].str.strip("['\"]")
df["discipline_principale_du_projet_c"] = df["discipline_principale_du_projet_c"].str.replace(";", ", ")

In [145]:
print(df.iloc[0])

name                                                                     Concrete Block Party
nom_de_la_structure_c                                                             Rara Woulib
presentation_synthetique_du_projet_c        "Riding Block Party" est une installation urba...
date_de_debut_c                                                     2024-05-25T12:00:00+00:00
date_de_fin_c                                                       2024-05-25T19:00:00+00:00
public_adresse_c                                                                 Tous publics
lieu_de_presentation_c                                              Place Villeneuve-Bargemon
adresse_c                                   Place Villeneuve - Bargemon, 13002 Marseille, ...
longitude_c                                                                 5.370179450940349
latitude_c                                                                 43.296730600000004
departement                                                 

On supprime `longitude_c`, `latitude_c` qui ne sont pas au bon format au profit de `geolocation.lon` et `geolocation.lat` (que l'on renomme). On supprime également `geolocation`.

In [146]:
df = df.drop(columns=["longitude_c", "latitude_c", "geolocation"])
df = df.rename(columns={"geolocation.lon": "longitude", "geolocation.lat": "latitude"})

On supprime également `departement` et `commune` qui sont redondants avec l'adresse.

In [147]:
df = df.drop(columns=["departement", "commune"])

Certains événement n'ont pas de géolocalisation, nous choisissons de les retirer du jeux données car nous ne pourrons pas les afficher correctement dans l'application.

In [149]:
df = df.dropna()

In [152]:
df.columns.to_list()

['name',
 'nom_de_la_structure_c',
 'presentation_synthetique_du_projet_c',
 'date_de_debut_c',
 'date_de_fin_c',
 'public_adresse_c',
 'lieu_de_presentation_c',
 'adresse_c',
 'tarif_du_projet_c',
 'veuillez_preciser_les_tarifs_du_projet_c',
 'instagram_link',
 'facebook_link',
 'twitter_link',
 'discipline_principale_du_projet_c',
 'longitude',
 'latitude']

In [154]:
df = df.rename(columns={
    "nom_de_la_structure_c": "nom_structure",
    "presentation_synthetique_du_projet_c": "presentation_projet",
    "date_de_debut_c": "date_debut",
    "date_de_fin_c": "date_fin",
    "public_adresse_c": "public",
    "lieu_de_presentation_c": "lieu_presentation",
    "adresse_c": "addresse",
    "tarif_du_projet_c": "tarif",
    "veuillez_preciser_les_tarifs_du_projet_c": "detail_tarif",
    "discipline_principale_du_projet_c": "discipline_projet",
})

Le dataset est prêt à être exporté.

In [156]:
# df.to_json("datasets/jop2024-evenements.json", orient="index", indent=4, date_format="iso")